# Analyse news from snopes



In [1]:
#Imports
import json
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from matplotlib.font_manager import json_load
from urllib.request import Request, urlopen

In [6]:
 ### Recolher os links que se pretende analisar
    

req = Request("https://www.snopes.com/fact-check/")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    if type (link.get('href')) == str and (len( link.get('href'))) >  50:  
        links.append(link.get('href'))


In [7]:
list_urls = links[:-1]
list_urls

['https://www.snopes.com/fact-check/musk-reinstate-trump-on-twitter/',
 'https://www.snopes.com/fact-check/the-onion-banned-on-twitter/',
 'https://www.snopes.com/fact-check/electric-bus-fire-kenya/',
 'https://www.snopes.com/fact-check/fbi-internet-slang/',
 'https://www.snopes.com/fact-check/hot-dog-under-microscope/',
 'https://www.snopes.com/fact-check/couple-divorce-beanie-babies/',
 'https://www.snopes.com/fact-check/florida-math-critical-race-theory/',
 'https://www.snopes.com/fact-check/herman-cain-newsmax/',
 'https://www.snopes.com/fact-check/man-rupture-lung-while-masturbating/',
 'https://www.snopes.com/fact-check/bath-and-body-works-infertility/',
 'https://www.snopes.com/fact-check/train-crash-metrolink/',
 'https://www.snopes.com/fact-check/catholic-church-opposed-street-lights/',
 'https://www.snopes.com/fact-check/did-snickers-remove-a-dick-vein/',
 'https://www.snopes.com/fact-check/florida-math-critical-race-theory/',
 'https://www.snopes.com/news/2022/04/20/verizon-

In [10]:
language = 'en'

def extractor (url):
    '''
    
        1) Extrair conteudo do url da noticia
        2) Identificar as componentes na noticia: alegaçao, avalicao, editor, url
        3) Guardar em dataset
        
        
    '''
    
    new = requests.get(url)
    soup = BeautifulSoup(new.content, 'html.parser')
    
    #Extract Title
    title = soup.find(class_='title').contents[0].strip('\n').strip('\t')

    #extract description
    for tag in soup.find_all("meta"):
        if tag.get("name", None) == "description":
            description = tag.get("content", None)
            
     #extract claim
    claim = soup.find(class_="claim-text card-body").contents[0].strip('\n').strip('\t')
            
            
    #extract classification         
    evaluation =  soup.find(class_='media-body d-flex flex-column align-self-center')
    for span in evaluation.find_all('span', recursive=False):
        classification = span.get("class", None)[1].replace("rating-label-", "")
        
    editor = url.split('.')[1]

     #extract data 
    for span in soup.find_all('time'):
        datatime = span.get("datetime", None)
       
    
    #  text = readNews.main(url, "text").encode('utf8').decode().split('\n')
    
    # Extract the text of the new
    text = []
    for data in soup.find(class_='single-body card-body rich-text').find_all("p"): 
        if data.get_text() == "Sources:":
            break
        text.append(data.get_text()) 

  
    return title , description, claim, text, classification, editor, datatime


    
def main ():
    for url in list_urls:
        title , description,  claim, text, classification, editor, datatime =  extractor (url)

        # Getting the current date and time
        dt = datetime.now()

        # getting the timestamp
        ts = datetime.timestamp(dt)

        obj = { "allegation": claim, "evaluation": classification , "nameNewspaper":editor , 'url': url, "language": language , "postText": text}
              
        #print(obj)
      
        with open("extractions/" + editor + "_extraction_" + str(ts)+ ".json", "w", encoding = 'utf8') as jsonFile:
            json.dump(obj, jsonFile, indent = 4, ensure_ascii=False)


In [11]:
main()

FileNotFoundError: [Errno 2] No such file or directory: 'extractions/snopes_extraction_1650643239.753576.json'